In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [2]:
df = pd.read_pickle('cleaned_df.pkl')

In [3]:
df.reset_index()

,index,title,text,link,author,label,clean_title,clean_text
0,0,Twitter Takedown Targets QAnon Accounts,“These accounts amplify and enable networked h...,https://www.nytimes.com/2020/07/21/technology/...,[Kate Conger],0,twitter takedown targets qanon accounts,accounts amplify enable networked harassment ...
1,1,Major Security Flaws Found in South Korea Quar...,“We were really in a hurry to make and deploy ...,https://www.nytimes.com/2020/07/21/technology/...,"[Choe Sang-Hun, Aaron Krolik, Raymond Zhong, N...",0,major security flaws found south korea quaran...,really hurry make deploy app quickly possible...
2,2,Tensions Between U.S. and China Rattle Investo...,"Luxuries do still exist during the pandemic, ‘...",https://www.nytimes.com/live/2020/07/22/busine...,"[Niraj Chokshi, Kevin Granville, Gillian Fried...",0,tensions u china rattle investors live busine...,luxuries still exist pandemic money united ai...
3,3,Coronavirus Threatens the Luster of Superstar ...,"From 1980 to 2018, the income per person in Ne...",https://www.nytimes.com/2020/07/21/business/ec...,[Eduardo Porter],0,coronavirus threatens luster superstar cities,income per person new york metropolitan area ...
4,4,A Company Backs a Cause. It Funds a Politician...,Walmart calls its employees “heroes” for putti...,https://www.nytimes.com/2020/07/21/business/de...,[Andrew Ross Sorkin],0,company backs cause funds politician gives,walmart calls employees heroes putting health...
...,...,...,...,...,...,...,...,...
790,790,Get ready for “The ‘M’ Word” by Jeff Foxworthy,Isn’t it strange when you can use a comedy pla...,https://www.usasupreme.com/get-ready-m-word-je...,"[Alex D., Alex D Is A Conservative Journalist,...",1,get ready word jeff foxworthy,strange use comedy platform speak truth yet p...
791,791,Somali Muslims Take Over Small Tennessee Town ...,Obama’s plan to bring refugees in our country ...,https://www.usasupreme.com/somali-muslims-take...,"[Natalie D., Natalie D. Is An American Conserv...",1,somali muslims take small tennessee town forc...,obama plan bring refugees country spread nati...
792,792,Somali Muslims Take Over Small Tennessee Town ...,Obama’s plan to bring refugees in our country ...,https://www.usasupreme.com/somali-muslims-take...,"[Natalie D., Natalie D. Is An American Conserv...",1,somali muslims take small tennessee town forc...,obama plan bring refugees country spread nati...
793,793,Obama Admin Funneled Billions Into Left-Wing A...,The leftists scream about taxpayer money for s...,https://www.usasupreme.com/obama-admin-funnele...,"[Alex D., Alex D Is A Conservative Journalist,...",1,obama admin funneled billions left wing activ...,leftists scream taxpayer money secret service...


In [4]:
# Preparing the target and predictors for modeling
# Keep the title and body text separated for different models

X_body_text = df['clean_text'].values
X_title_text = df['clean_title'].values
y = df['label'].values

# create tf idf vector

In [5]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df= 0.85, min_df= 0.01)

In [6]:
X_body_tfidf = tfidf.fit_transform(X_body_text)
X_title_tfidf = tfidf.fit_transform (X_title_text)

In [7]:
indices = df.index.values

# 80% of articles for training, 20% for testing from X and y

In [8]:
X_body_tfidf_train, X_body_tfidf_test, y_body_train, y_body_test, \
indices_body_train, indices_body_test = train_test_split(X_body_tfidf, y, indices, test_size = 0.2, random_state=42)

In [9]:
df.loc[indices_body_train].groupby('label').agg('count')  #training data states 

,title,text,link,author,clean_title,clean_text
label,,,,,,
0,468,468,468,468,468,468
1,168,168,168,168,168,168


In [10]:
df.loc[indices_body_test].groupby('label').agg('count')   #test data states

,title,text,link,author,clean_title,clean_text
label,,,,,,
0,111,111,111,111,111,111
1,48,48,48,48,48,48


# Import naive bayes classifier

In [11]:
from sklearn.naive_bayes import MultinomialNB

In [12]:
nb_body = MultinomialNB()        #initialize nb classifier

fit training data into classifier

In [13]:
nb_body.fit(X_body_tfidf_train, y_body_train)

MultinomialNB()

prediction 

In [14]:
y_body_train_pred = nb_body.predict(X_body_tfidf_train)

In [15]:
print('Naive Bayes In Sample F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(y_body_train, y_body_train_pred, average='macro')*100 ))
print ('Accuracy score {:.4}%'.format(accuracy_score(y_body_train, y_body_train_pred)*100))

Naive Bayes In Sample F1 and Accuracy Scores:
F1 score 87.96%
Accuracy score 91.67%
